# Capstone Project - The Battle of Neighborhoods

## Introduction 

London house prices have soared more than in any other part of the UK since the 2007-08 financial crisis.

In fact, it is now so expensive buying an house in London that many first-time buyers have given up and are sticking to renting. 

The business problem this project aims to solve is to identify which London postcode district provides the best __value-for-money__ for a two-bedrooms home.

## Data

The following sources have been analysed in order to identify and collect the necessary information and data:
1. [Private rental market in London – July 2018 to June 2019](https://www.gov.uk/government/publications/private-rental-market-in-london-july-2018-to-june-2019)
UK Government official statistics on the private rental market in London covering July 2018 to June 2019 by region, borough and postcode district.
2. [Foursquare API](https://developer.foursquare.com)
Foursquare database to collect information on venue categories, tips, location data, venue ratings.
3. [Complete list of London postal districts with their longitude and latitude and OS coordinates](https://www.doogal.co.uk/london_postcodes.php)
